<a href="https://colab.research.google.com/github/yg-renaissance/Generate-Lyrics/blob/master/baseline_lyrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!
import os
# 파일을 읽기모드로 열어 봅니다.
file_path = './aiffel/lyricist/data/shakespeare.txt'
with open(file_path, "r") as f:
    raw_corpus = f.read().splitlines()   # 텍스트를 라인 단위로 끊어서 list 형태로 읽어옵니다.
print(raw_corpus[:9])    # 앞에서부터 10라인만 화면에 출력해 볼까요?

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


In [ ]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.
    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


In [ ]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \\1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    return sentence
print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence <end>


In [ ]:
corpus = []
for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    corpus.append(preprocess_sentence(sentence))
corpus[:10]

['<start> before we proceed any further hear me speak <end>',
 '<start> speak speak <end>',
 '<start> you are all resolved rather to die than to famish <end>',
 '<start> resolved resolved <end>',
 '<start> first you know caius marcius is chief enemy to the people <end>',
 '<start> we know t we know t <end>',
 '<start> let us kill him and we ll have corn at our own price <end>',
 '<start> is t a verdict <end>',
 '<start> no more talking on t let it be done away away <end>',
 '<start> one word good citizens <end>']

In [ ]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.
    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   
    # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.
    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    print(tensor,tokenizer)
    return tensor, tokenizer
tensor, tokenizer = tokenize(corpus)

[[   2  139   36 ...    0    0    0]
 [   2  106  106 ...    0    0    0]
 [   2    9   46 ...    0    0    0]
 ...
 [   2  145 4549 ...    0    0    0]
 [   2   30   67 ...    0    0    0]
 [   2  941   30 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x0000024F19323370>


In [ ]:
print(tensor[:3, :10])

[[   2  139   36  929  136  587  120   20  106    3]
 [   2  106  106    3    0    0    0    0    0    0]
 [   2    9   46   39 1197  312    7  197   70    7]]


In [ ]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : the
5 : and
6 : i
7 : to
8 : of
9 : you
10 : my


In [ ]:
src_input = tensor[:, :-1]  
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
print(src_input[0])
print(tgt_input[0])

[  2 139  36 929 136 587 120  20 106   3   0   0   0   0   0   0   0]
[139  36 929 136 587 120  20 106   3   0   0   0   0   0   0   0   0]


In [ ]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE
VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 17), (256, 17)), types: (tf.int32, tf.int32)>

In [ ]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        return out
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [ ]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 17, 7001), dtype=float32, numpy=
array([[[-1.14655471e-04, -1.78517766e-05,  3.08029848e-06, ...,
         -1.21848148e-04,  3.87954060e-08,  8.63766181e-05],
        [-1.49462488e-04, -1.56674738e-04,  1.15113238e-04, ...,
         -5.35192084e-05, -4.58535505e-05, -1.65066478e-04],
        [-7.24038255e-05, -1.31244713e-04,  3.23831482e-04, ...,
         -2.00165989e-04,  8.45187460e-05, -3.54463176e-04],
        ...,
        [-1.55866286e-03, -2.94863456e-03,  4.95855045e-03, ...,
          2.80671031e-03, -1.70373928e-03, -1.44338305e-03],
        [-1.76224019e-03, -3.24353715e-03,  5.50625287e-03, ...,
          2.94659287e-03, -1.76372402e-03, -1.27903011e-03],
        [-1.95621233e-03, -3.50603927e-03,  5.98925725e-03, ...,
          3.05174454e-03, -1.81078666e-03, -1.10585801e-03]],

       [[-1.14655471e-04, -1.78517766e-05,  3.08029848e-06, ...,
         -1.21848148e-04,  3.87954060e-08,  8.63766181e-05],
        [-3.61169514e-04,  1.48726147e-04, -8.

In [ ]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1792256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  7176025   
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)
model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
93/93 [==============================] - 282s 3s/step - loss: 4.5859
Epoch 2/30
93/93 [==============================] - 296s 3s/step - loss: 3.0296
Epoch 3/30
93/93 [==============================] - 295s 3s/step - loss: 2.9674
Epoch 4/30
93/93 [==============================] - 297s 3s/step - loss: 2.8915
Epoch 5/30
93/93 [==============================] - 296s 3s/step - loss: 2.8345
Epoch 6/30
93/93 [==============================] - 296s 3s/step - loss: 2.7766
Epoch 7/30
93/93 [==============================] - 300s 3s/step - loss: 2.7206
Epoch 8/30
30/93 [========>.....................] - ETA: 3:28 - loss: 2.6767

In [ ]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]
    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 
        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
																 tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break
    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "
    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [ ]:
generate_text(model, tokenizer, init_sentence="<start> he")